In [1]:
!pip install duckdb==0.5.1
!pip install pytorch_tabular[extra]
!pip install torch
!pip install lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 12.1 MB/s eta 0:00

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.0/546.0 KB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 7.5 MB/s eta 0:00:00
 

In [1]:
import sys,os
project_path = "/content/drive/MyDrive/Work/quantitative_research/"
#database_path = "/content/drive/MyDrive/Work/trading_updated/database/development/develop"

sys.path.append(project_path)
#from featurewiz import featurewiz
import random
import lightgbm as lgb
import numpy as np
import sklearn.metrics as mt
import duckdb
import pandas as pd
import data.utils.duckdb_utils as ddu
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import os

In [2]:
database_path = "/content/drive/Othercomputers/My Mac/google_drive/database/production/banknifty_final1.duckdb"
db_connection = duckdb.connect(database=database_path , read_only=False)
train_table = 'banknifty.train'
val_table = 'banknifty.validation'
buffer_table = 'banknifty.buffer'

In [3]:
data_suffix = 'improved_one'
train_feature_table = f'banknifty.train_feature_{data_suffix}'
train_feature_info_table = f'banknifty.train_feature_info_{data_suffix}'
train_feature_selection_table = f'banknifty.train_feature_selection_{data_suffix}'
train_training_info_table = f'banknifty.train_training_info_{data_suffix}'
train_tuning_info_table = f'banknifty.train_tuning_info_{data_suffix}'
ohlcraw_table = 'banknifty.ohlc_raw_data'

train_feature_selection_table_df = ddu.load_table_df(db_connection,table_name=train_feature_selection_table,
                                             column_names=None,filter=None,load_sql=None)



In [39]:
target = 'label__minus_15_1min_close__label_generator_4class'
cols = train_feature_selection_table_df[train_feature_selection_table_df['label_name']==target]['feature_name'].tolist()

In [40]:
len(cols)

366

In [6]:
sql = f"Select count(*) as cnt, time_split from {train_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train_cols = [col for col in a['time_split'].tolist() if 'train' in str(col)]

sql = f"Select count(*) as cnt, time_split from {val_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
val_cols = [col for col in a['time_split'].tolist() if 'validation' in str(col)]

sql = f"Select count(*) as cnt, time_split from {buffer_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
buffer_cols = [col for col in a['time_split'].tolist() if 'buffer' in str(col)]


In [7]:
len(train_cols)

10

In [8]:
from fastai.tabular.core import df_shrink_dtypes,df_shrink,add_datepart

In [16]:
cat_cols = ['timestampYear',
 'timestampMonth',
 'timestampWeek',
 'timestampDay',
 'timestampDayofweek',
 'timestampDayofyear',
 'timestampIs_month_end',
 'timestampIs_month_start',
 'timestampIs_quarter_end',
 'timestampIs_quarter_start',
 'timestampIs_year_end',
 'timestampIs_year_start',
 'timestampElapsed']

In [29]:
sql = f'''select {",".join(cols)},{target},timestamp from {train_table} where time_split in 
('{"','".join(random.sample(train_cols, 5)) }') and
{target} != 'unknown' 
'''

train = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train = add_datepart (train, field_name ='timestamp', prefix=None, drop=True, time=False)

train= df_shrink(train)
for c in cat_cols:
  train[c] = train[c].astype(str)
  train[c] = train[c].astype('category')

In [30]:
train[cat_cols].head()

,timestampYear,timestampMonth,timestampWeek,timestampDay,timestampDayofweek,timestampDayofyear,timestampIs_month_end,timestampIs_month_start,timestampIs_quarter_end,timestampIs_quarter_start,timestampIs_year_end,timestampIs_year_start,timestampElapsed
0,2020,6,23,2,1,154,False,False,False,False,False,False,1591099100.0
1,2020,6,23,2,1,154,False,False,False,False,False,False,1591099100.0
2,2020,6,23,2,1,154,False,False,False,False,False,False,1591099100.0
3,2020,6,23,2,1,154,False,False,False,False,False,False,1591099300.0
4,2020,6,23,2,1,154,False,False,False,False,False,False,1591099300.0


In [31]:
train.shape

(246494, 380)

In [32]:
sql = f'''select {",".join(cols)},{target},timestamp  from {val_table} where time_split in ('{"','".join(random.sample(val_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
valid = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
valid = add_datepart (valid, field_name ='timestamp', prefix=None, drop=True, time=False)

valid= df_shrink(valid)
for c in cat_cols:
  valid[c] = valid[c].astype(str)
  valid[c] = valid[c].astype('category')
valid.shape

select ROLLSTT_120_2_MEANDIFF_close,ROLLSTT_45_2_MAXMIN_close20,ROLLRNK_EMA_15_ema_close_15_None,ROLLDIFF_0_0_super_diff13,ROLLRNK_EMA_45_ema_close10_60_None,ROLLSTT_120_2_MAXMIN_close,TRUERANGE_120_hlc,ROLLSTT_30_2_MAXMIN_close,ROLLSTT_15_2_MAXMIN_close,ROLLPCM_DCL_60_60_don4_30_None,ROLLRNK_EMA_45_ema_close10_15_None,ROLLPCM_DCU_30_30_don2_30_None,ROLLRNK_EMA_120_ema_close20_15_None,DIFFCOL_DCU_20_20_hl50_close,ROLLPCM_DCU_60_60_don4_60_None,ROLLSTT_60_2_MAXMIN_close20,ROLLPCM_SUPERTs_45_30_supertrend_close_30_None,ROLLPCM_DCL_30_30_don2_30_None,ROLLPCM_DCU_240_240_don5_15_None,DIFFCOL_DCL_60_60_don4_close,ADX_30_adx14,ROLLBIN_100_0_True_5_close,ROLLPCM_DCL_120_120_don9_60_None,ROLLPCM_SUPERTs_15_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_60_None,ROLLPCM_DCM_30_30_don7_30_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_15_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_60_None,ROLLPCM_DCM_120_120_don14_15_None,ROLLPCM_DCL_240_240_don5_60_None,RO

(74904, 380)

In [43]:
sql = f'''select {",".join(cols)},{target},timestamp from {buffer_table} where time_split in ('{"','".join(random.sample(buffer_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
test = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
test = add_datepart (test, field_name ='timestamp', prefix=None, drop=True, time=False)

test= df_shrink(test)
for c in cat_cols:
  test[c] = test[c].astype(str)
  test[c] = test[c].astype('category')
test.shape

select ROLLSTT_120_2_MEANDIFF_close,ROLLSTT_45_2_MAXMIN_close20,ROLLRNK_EMA_15_ema_close_15_None,ROLLDIFF_0_0_super_diff13,ROLLRNK_EMA_45_ema_close10_60_None,ROLLSTT_120_2_MAXMIN_close,TRUERANGE_120_hlc,ROLLSTT_30_2_MAXMIN_close,ROLLSTT_15_2_MAXMIN_close,ROLLPCM_DCL_60_60_don4_30_None,ROLLRNK_EMA_45_ema_close10_15_None,ROLLPCM_DCU_30_30_don2_30_None,ROLLRNK_EMA_120_ema_close20_15_None,DIFFCOL_DCU_20_20_hl50_close,ROLLPCM_DCU_60_60_don4_60_None,ROLLSTT_60_2_MAXMIN_close20,ROLLPCM_SUPERTs_45_30_supertrend_close_30_None,ROLLPCM_DCL_30_30_don2_30_None,ROLLPCM_DCU_240_240_don5_15_None,DIFFCOL_DCL_60_60_don4_close,ADX_30_adx14,ROLLBIN_100_0_True_5_close,ROLLPCM_DCL_120_120_don9_60_None,ROLLPCM_SUPERTs_15_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_60_None,ROLLPCM_DCM_30_30_don7_30_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_15_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_60_None,ROLLPCM_DCM_120_120_don14_15_None,ROLLPCM_DCL_240_240_don5_60_None,RO

(74847, 380)

In [44]:
cols = cols+cat_cols

In [36]:
model_params= {'metric': 'multi_logloss', 'learning_rate': 0.01, 'objective':'multiclass',
               'lambda_l1': 0.0023608644618906675, 'num_iterations':1000,
               'lambda_l2': 3.2269885945997864e-07, 
               'num_leaves': 45, "verbose":0,
               "is_unbalance": True,
               'feature_fraction': 0.4086060680211939, 
               'bagging_fraction': 0.4660039780815337, 'bagging_freq': 1, 'min_child_samples': 25}
model_fit_params={"early_stopping_rounds":10, "verbose":100, "eval_metric":None}
model_fit_params.update({'X':train[cols]})
model_fit_params.update({'y':train[target]})
model_fit_params.update({'categorical_feature':cat_cols})
model_fit_params.update({'eval_set':[(valid[cols], valid[target]), 
                                          (train[cols], train[target])]})
model_fit_params.update({'eval_names':['valid','train']})
model = lgb.LGBMClassifier(**model_params)
    

In [37]:
model.fit(**model_fit_params)

/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:2065: UserWarning: Using ca

[LightGBM] [Warning] feature_fraction is set=0.4086060680211939, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4086060680211939
[LightGBM] [Warning] lambda_l1 is set=0.0023608644618906675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0023608644618906675
[LightGBM] [Warning] bagging_fraction is set=0.4660039780815337, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4660039780815337
[LightGBM] [Warning] lambda_l2 is set=3.2269885945997864e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2269885945997864e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.4086060680211939, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4086060680211939
[LightGBM] [Warning] lambda_l1 is set=0.0023608644618906675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0023608644618906675
[LightGBM] [Warni

/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] feature_fraction is set=0.4086060680211939, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4086060680211939
[LightGBM] [Warning] lambda_l1 is set=0.0023608644618906675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0023608644618906675
[LightGBM] [Warning] bagging_fraction is set=0.4660039780815337, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4660039780815337
[LightGBM] [Warning] lambda_l2 is set=3.2269885945997864e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2269885945997864e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] feature_fraction is set=0.4086060680211939, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4086060680211939
[LightGBM] [Warning] lambda_l1 is set=0.0023608644618906675, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0023608644618906675
[LightGBM] [Warning] bagging_fraction is set=0.4660039780815337, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4660039780815337
[LightGBM] [Warning] lambda_l2 is set=3.2269885945997864e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2269885945997864e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[100]	valid's multi_logloss: 0.723793	train's multi_logloss: 0.82357
[200]	valid's multi_logloss: 0.713292	train's multi_logloss: 0.779922


LGBMClassifier(bagging_fraction=0.4660039780815337, bagging_freq=1,
               feature_fraction=0.4086060680211939, is_unbalance=True,
               lambda_l1=0.0023608644618906675,
               lambda_l2=3.2269885945997864e-07, learning_rate=0.01,
               metric='multi_logloss', min_child_samples=25,
               num_iterations=1000, num_leaves=45, objective='multiclass',
               verbose=0)

In [45]:
pred_probs = model.predict_proba(test[cols])
preds = model.predict(test[cols]) 

In [46]:
import sklearn.metrics as mt
c = mt.classification_report(test[target],preds)
c1 = mt.confusion_matrix(test[target],preds)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
print(c)

              precision    recall  f1-score   support

        call       0.00      0.00      0.00      8461
     neutral       0.77      1.00      0.87     57385
         put       0.00      0.00      0.00      9001

    accuracy                           0.77     74847
   macro avg       0.26      0.33      0.29     74847
weighted avg       0.59      0.77      0.67     74847



In [ ]:
data_config = DataConfig(
    target=[target], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=cols)

trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
#     accelerator="cpu"
)
optimizer_config = OptimizerConfig()

# DEPRECATED
# prediction head is defined separately now and head & head_config will be made
# mandatory in future releases
# model_config = CategoryEmbeddingModelConfig(
#     task="classification",
#     layers="1024-512-512",  # Number of nodes in each layer
#     activation="LeakyReLU", # Activation between each layers
#     learning_rate = 1e-3,
#     metrics=["f1","accuracy"], 
#     metrics_params=[{"num_classes":2},{}]
# )

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

loss = tt.AveragePrecision(task="multiclass", num_classes=3, average=None)

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3,
    loss = loss,
    metrics=["f1_score","accuracy"], 
    metrics_params=[{"num_classes":3},{}]
)

In [ ]:
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy
sampler = get_balanced_sampler(train[target].values.ravel())
weighted_loss = get_class_weighted_cross_entropy(train[target].values.ravel(), mu=0.04)


In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-03-25 18:00:06,386 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
tabular_model.fit(train=train, validation=valid, 
                  #train_sampler=sampler, 
                  loss=weighted_loss)


INFO:lightning_lite.utilities.seed:Global seed set to 42
2023-03-25 18:00:07,086 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-03-25 18:00:07,449 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
2023-03-25 18:00:52,380 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-03-25 18:00:52,480 - {pytorch_tabular.tabular_model:264} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False,

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=91` reached.
INFO:pytorch_lightning.tuner.lr_finder:LR finder stopped early after 91 steps due to diverging loss.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.000363078054770101
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_6723d162-f4be-486b-bbbe-e46f50ad8e6d.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_6723d162-f4be-486b-bbbe-e46f50ad8e6d.ckpt
2023-03-25 18:01:14,069 - {pytorch_tabular.tabular_model:560} - INFO - Suggested LR: 0.000363078054770101. For plot and detailed analysis, use `find_learning_rate` method.
INFO:pytorch_tabular.tabular_model:Suggested LR: 0.000363078054770101. For plot and detailed analysis, use `find_learning_rate` method.
2023-03-25 18:01:14,073 - {pytorch_tabular.tabular_model:566} - INFO - Training Started
INFO:pytorch_tabular.tabular_mode

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ custom_loss      │ CrossEntropyLoss          │      0 │
│ 1 │ _backbone        │ CategoryEmbeddingBackbone │  1.2 M │
│ 2 │ _embedding_layer │ Embedding1dLayer          │    732 │
│ 3 │ head             │ LinearHead                │  1.5 K │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 1.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.2 M                                                                                                
Total estimated model params size (MB): 4

Output()

2023-03-25 18:05:32,497 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
INFO:pytorch_tabular.tabular_model:Training the model completed
2023-03-25 18:05:32,502 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model
INFO:pytorch_tabular.tabular_model:Loading the best model


In [ ]:
sql = f'''select {",".join(cols)},{target} from {buffer_table} where time_split in ('{"','".join(random.sample(buffer_cols, 5)) }') and
{target} != 'unknown' 
'''
test = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
test= df_shrink(test)
test.shape

(64141, 367)

In [ ]:
result = tabular_model.evaluate(test)


Output()

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.4726306200027466     │
│       test_f1_score       │    0.4726306200027466     │
│         test_loss         │    1.0384968519210815     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
q = tabular_model.predict(test)

Output()

In [ ]:
q[['label__minus_15_1min_close__label_generator_4class','prediction']]

,label__minus_15_1min_close__label_generator_4class,prediction
0,put,put
1,put,call
2,put,call
3,put,call
4,put,call
...,...,...
64136,put,put
64137,put,put
64138,put,neutral
64139,put,neutral


In [ ]:
q['prediction'].value_counts()

neutral    36686
put        14284
call       13171
Name: prediction, dtype: int64

In [ ]:
q['label__minus_15_1min_close__label_generator_4class'].value_counts()

neutral    28721
call       17744
put        17676
Name: label__minus_15_1min_close__label_generator_4class, dtype: int64

In [ ]:
tabular_model.summary()

   | Name                                    | Type                      | Params
---------------------------------------------------------------------------------------
0  | custom_loss                             | CrossEntropyLoss          | 0     
1  | _backbone                               | CategoryEmbeddingBackbone | 1.2 M 
2  | _backbone.linear_layers                 | Sequential                | 1.2 M 
3  | _backbone.linear_layers.0               | Linear                    | 375 K 
4  | _backbone.linear_layers.1               | LeakyReLU                 | 0     
5  | _backbone.linear_layers.2               | Linear                    | 524 K 
6  | _backbone.linear_layers.3               | LeakyReLU                 | 0     
7  | _backbone.linear_layers.4               | Linear                    | 262 K 
8  | _backbone.linear_layers.5               | LeakyReLU                 | 0     
9  | _embedding_layer                        | Embedding1dLayer          | 732   
10 | _embe

In [ ]:
print(c)

              precision    recall  f1-score   support

        call       0.35      0.26      0.29     17744
     neutral       0.56      0.72      0.63     28721
         put       0.37      0.30      0.33     17676

    accuracy                           0.47     64141
   macro avg       0.42      0.42      0.42     64141
weighted avg       0.45      0.47      0.45     64141



In [ ]:
c1

array([[ 4553,  8356,  4835],
       [ 3946, 20544,  4231],
       [ 4672,  7786,  5218]])